<a href="https://colab.research.google.com/github/lail-lei/nlp-cupcakes/blob/main/fast_keyword_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [166]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer 
import math
import re
from textblob import TextBlob as tb
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from google.colab import files

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [167]:
# our data from csv
df = pd.read_csv('https://raw.githubusercontent.com/lail-lei/nlp-cupcakes/main/cupcakes.csv')

In [168]:
df = df.iloc[:, 0:7]
df

,Title,Yield,Ingredients,Steps,URL,Tags,Description
0,Baileys & Coffee Cupcakes,14 cupcakes,"[{'amount': '1', 'text': '1 and 3/4 cups cake ...","['Preheat the oven to 350 F (177 C).', 'Line a...",https://sallysbakingaddiction.com/baileys-coff...,"[' cupcakes', ' desserts', ' st patricks day',...",NaN
1,Guinness Chocolate Cupcakes with Mocha Guinnes...,16 cupcakes,"[{'amount': '1', 'text': 'one 12 ounce bottle ...","['In a small saucepan, bring the Guinness to a...",https://sallysbakingaddiction.com/guinness-cho...,"[' cupcakes', ' desserts', ' st patricks day',...",NaN
2,Molten Cookie Dough Cupcakes,12 cupcakes,"[{'amount': '2', 'text': '2 and 1/4 cups heat ...","[' In a medium bowl, whisk the heat treated fl...",https://sallysbakingaddiction.com/molten-cooki...,"[' cupcakes', ' desserts', ' all purpose flour...",NaN
3,Yellow Birthday Cupcakes (Recipe Solved!),12-14 cupcakes,"[{'text': '1 and 3/4 cups cake flour '}, {'tex...","['Preheat the oven to 350 F (177 C).', 'Line a...",https://sallysbakingaddiction.com/yellow-birth...,"[' birthday', ' cupcakes', ' seasonal', ' butt...",NaN
4,Confetti Sprinkle Cupcakes,14 cupcakes,"[{'amount': '1', 'text': '1 and 3/4 cups cake ...","['Preheat the oven to 350 F (177 C).', 'Line a...",https://sallysbakingaddiction.com/confetti-cup...,"[' birthday', ' cupcakes', ' spring recipes', ...",NaN
...,...,...,...,...,...,...,...
1552,"Wheat Germ and Ginger, Wasabi Cream, and Tuna ...",12 cupcakes,"[{'text': '3/4 cup all purpose flour'}, {'text...",['For the cupcakes Preheat the oven to 325 deg...,https://www.foodnetwork.com/recipes/wheat-germ...,"['Mixer Recipes', 'Cupcake', 'Sugar', 'Fish', ...",NaN
1553,Bunny Face,1 cupcake,"[{'text': '1 Vanilla Cupcake recipe follows'},...",['Dip the surface of the cupcake into the Deep...,https://www.foodnetwork.com/recipes/bunny-face...,"['Easy Dessert Recipes', 'Dessert', 'Easter Cu...",NaN
1554,Winning Recipe Mini Coconut Cupcakes with Popp...,48 cupcakes,"[{'text': '1 cup butter'}, {'text': '1 cup sug...",['For the toffee Line a baking sheet with a si...,https://www.foodnetwork.com/recipes/winning-re...,"['Cupcake', 'Fruit', 'Raisin Recipes', 'Coconu...",NaN
1555,South Beach Wine and Food Festival Cupcakes,12 cupcakes,[{'text': '4 ounces red wine such as cabernet ...,['For the cupcakes Preheat the oven to 350 deg...,https://www.foodnetwork.com/recipes/south-beac...,"['Chocolate Cupcake', 'Cupcake', 'Baking', 'De...",NaN


In [169]:
def process (text):
  all_stopwords = stopwords.words('english')
  # recipe_specific = [ 'amount', 'unit', 'title', 'yield', 'ingredients', 'text', 'steps', 'url', 'description', 'tags', "tsp", 'cup', 'cups', 'tbsp', "f", "c", "( c )"
  #                    "teaspoon", 'teaspoons', 'tablespoon', 'tablespoons', 'lbs', 'grams', 'gm', "gms", "kg", "kilograms", "kilo", "kgs"]

  recipe_specific = ["title", "ingredients", "amount", "unit", "text", "teaspoon", "teaspoons", "tsp", "cup", "cups", "tablespoon", "tablespoons", "tbsp", "tb", 
                     "grams", "gram", "g", "gm", "gms", "pounds", "lbs", "pounds", "pound", "kilogram", "kg", "kilograms", "kgs", "kilo", "kilos", "milliliter", "ml", 
                     "pinch", "pinches", "dash", "dashes", "steps", "tags", "description", "f", "c", "yield", "tin", "muffin tin", "billington", "nielson massey", "allinson"]
  all_stopwords.extend(recipe_specific)
  text = re.sub(r"[^a-zA-Z]+", ' ', text).lower().strip()
  tokens = word_tokenize(text)
  lemmatizer = WordNetLemmatizer()
  lemmatized = [lemmatizer.lemmatize(w) for w in tokens]
  removing_custom_words = [words for words in lemmatized if not words in all_stopwords]
  return " ".join(removing_custom_words)

In [170]:
df["processed"] = df.apply(lambda x: process(x.to_json()), axis=1)

In [171]:
df["processed"][700]

'apple gingerbread bran muffin muffin ingredient oat bran plain nonfat greek yogurt nonfat milk vanilla extract whole wheat gluten free flour ground ginger ground cinnamon baking powder salt coconut oil unsalted butter melted cooled slightly large egg room temperature pure maple syrup molasses diced apple step preheat oven lightly coat muffin nonstick cooking spray medium bowl stir together oat bran yogurt milk vanilla whisk together flour ginger cinnamon baking powder salt separate bowl third bowl whisk together coconut oil egg stir maple syrup molasses thoroughly mix bran mixture add flour mixture stirring incorporated fold diced apple divide batter prepared muffin bake minute toothpick inserted center come clean cool pan minute carefully turning onto wire rack url http amyshealthybaking com blog apple gingerbread bran muffin tag baked bliss breakfast clean eating fruit gluten free low fat muffin vegetarian apple autumn baking bran bran muffin breakfast brunch clean eating fall food 

In [172]:
docs = df["processed"]

In [173]:
# sklearn's tfidf vectorizer to create matrix
tfIdfVectorizer = TfidfVectorizer(use_idf=True)
tfIdf = tfIdfVectorizer.fit_transform(docs)

# array to hold keywords
keywords = []

# get keywords for each document
for i in range(1557):
  matrix = pd.DataFrame(tfIdf[i].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
  matrix = matrix.sort_values('TF-IDF', ascending=False)
  matrix = matrix[:5]
  keywords.append([*matrix.index])
 

In [174]:
df["keywords"] = keywords

In [175]:
df.to_csv('cupcakes_keywords.csv') 
files.download('cupcakes_keywords.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>